In [1]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "trl<0.9.0" xformers

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-l4vj8w8_/unsloth_2a8fdfe9c3884baebf21423de8229caa
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-l4vj8w8_/unsloth_2a8fdfe9c3884baebf21423de8229caa
  Resolved https://github.com/unslothai/unsloth.git to commit 4af624557fbcc14e248daeb9709ce5a81c3070ca
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.3/208.3 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.7/131.7 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.7/564.7 kB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.6/213.6 kB 20.8 MB/s eta 0:

In [1]:
import unsloth
from unsloth import FastLanguageModel
import torch

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [5]:
max_seq_length = 2048
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/codellama-7b-bnb-4bit",
    max_seq_length=max_seq_length,
    dtype=None,
    load_in_4bit=True,
)

==((====))==  Unsloth 2025.9.5: Fast Llama patching. Transformers: 4.56.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/3.87G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/155 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/539 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [6]:
model = FastLanguageModel.get_peft_model(
    model,
    r=32,
    target_modules=['q_proj', 'k_proj', 'v_proj','o_proj', 'up_proj', 'down_proj', 'gate_proj'],
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    use_gradient_checkpointing='unsloth',
    use_rslora=False,
    loftq_config=None,
    random_state=3407,
)

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.9.5 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [7]:
model.print_trainable_parameters()

trainable params: 79,953,920 || all params: 6,818,500,608 || trainable%: 1.1726


In [11]:
from datasets import load_dataset
dataset = load_dataset("google/code_x_glue_ct_code_to_text","python", split="train[:3000]")

python/train-00000-of-00002.parquet:   0%|          | 0.00/144M [00:00<?, ?B/s]

python/train-00001-of-00002.parquet:   0%|          | 0.00/147M [00:00<?, ?B/s]

python/validation-00000-of-00001.parquet:   0%|          | 0.00/16.7M [00:00<?, ?B/s]

python/test-00000-of-00001.parquet:   0%|          | 0.00/18.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/251820 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13914 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/14918 [00:00<?, ? examples/s]

In [12]:
dataset

Dataset({
    features: ['id', 'repo', 'path', 'func_name', 'original_string', 'language', 'code', 'code_tokens', 'docstring', 'docstring_tokens', 'sha', 'url'],
    num_rows: 3000
})

In [13]:
def format_code_docs(examples):
    """Simple formatting: code -> docstring generation"""
    texts = []

    for i in range(len(examples["code"])):
        code = examples["code"][i]
        docstring = examples["docstring"][i]

        # Simple prompt format
        text = f"""<s>[INST] Generate a docstring for this Python code:

{code}
[/INST]
{docstring}</s>"""

        texts.append(text)

    return {"text": texts}

In [14]:
formatted_dataset = dataset.map(
    format_code_docs,
    batched=True,
    remove_columns=dataset.column_names,
    desc="Formatting for code documentation"
)

Formatting for code documentation:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [15]:
formatted_dataset

Dataset({
    features: ['text'],
    num_rows: 3000
})

In [17]:
print(formatted_dataset['text'][0])

<s>[INST] Generate a docstring for this Python code:

def settext(self, text, cls='current'):
        """Set the text for this element.

        Arguments:
            text (str): The text
            cls (str): The class of the text, defaults to ``current`` (leave this unless you know what you are doing). There may be only one text content element of each class associated with the element.
        """
        self.replace(TextContent, value=text, cls=cls)
[/INST]
Set the text for this element.

        Arguments:
            text (str): The text
            cls (str): The class of the text, defaults to ``current`` (leave this unless you know what you are doing). There may be only one text content element of each class associated with the element.</s>


In [19]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    warmup_steps=5,
    max_steps=100,
    learning_rate=2e-4,
    fp16=not torch.cuda.is_bf16_supported(),
    bf16=torch.cuda.is_bf16_supported(),
    logging_steps=10,
    optim="adamw_8bit",
    weight_decay=0.01,
    lr_scheduler_type="linear",
    seed=3407,
    output_dir="codellama_docs",
    save_strategy="no",
)

In [20]:
from trl import SFTTrainer
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=formatted_dataset,
    args=training_args,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
)

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [21]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 3,000 | Num Epochs = 1 | Total steps = 100
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 79,953,920 of 6,818,500,608 (1.17% trained)
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: abeshith (abeshith-dr-m-g-r-educational-and-research-institute) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
10,1.057100
20,0.850700
30,0.765500
40,0.738000
50,0.711500
60,0.745400
70,0.741700
80,0.741000
90,0.733400
100,0.773900


TrainOutput(global_step=100, training_loss=0.7858270740509034, metrics={'train_runtime': 1181.7826, 'train_samples_per_second': 0.677, 'train_steps_per_second': 0.085, 'total_flos': 1.771241732702208e+16, 'train_loss': 0.7858270740509034, 'epoch': 0.26666666666666666})

In [22]:
model.save_pretrained("codellama_documentation")
tokenizer.save_pretrained("codellama_documentation")

('codellama_documentation/tokenizer_config.json',
 'codellama_documentation/special_tokens_map.json',
 'codellama_documentation/tokenizer.model',
 'codellama_documentation/added_tokens.json',
 'codellama_documentation/tokenizer.json')

In [ ]:
FastLanguageModel.for_inference(model)

In [24]:
def generate_docstring(code):
    """Generate docstring for Python code - SIMPLE VERSION"""

    prompt = f"""<s>[INST] Generate a docstring for this Python code:

{code}
[/INST]"""

    inputs = tokenizer([prompt], return_tensors="pt").to("cuda")

    outputs = model.generate(
        **inputs,
        max_new_tokens=200,
        temperature=0.3,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
    )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    docstring = response.split("[/INST]")[-1].strip()

    return docstring

In [25]:
test_codes = [
    """def bubble_sort(arr):
    n = len(arr)
    for i in range(n):
        for j in range(0, n-i-1):
            if arr[j] > arr[j+1]:
                arr[j], arr[j+1] = arr[j+1], arr[j]
    return arr""",

    """def fibonacci(n):
    if n <= 1:
        return n
    return fibonacci(n-1) + fibonacci(n-2)""",

    """def binary_search(arr, target):
    left, right = 0, len(arr) - 1
    while left <= right:
        mid = (left + right) // 2
        if arr[mid] == target:
            return mid
        elif arr[mid] < target:
            left = mid + 1
        else:
            right = mid - 1
    return -1"""
]

print("🧪 TESTING CODE DOCUMENTATION GENERATION")
print("=" * 50)

for i, code in enumerate(test_codes, 1):
    print(f"\n📝 Test {i}:")
    print("Code:")
    print(code)
    print("\nGenerated Documentation:")
    doc = generate_docstring(code)
    print(doc)
    print("-" * 40)

🧪 TESTING CODE DOCUMENTATION GENERATION

📝 Test 1:
Code:
def bubble_sort(arr):
    n = len(arr)
    for i in range(n):
        for j in range(0, n-i-1):
            if arr[j] > arr[j+1]:
                arr[j], arr[j+1] = arr[j+1], arr[j]
    return arr

Generated Documentation:
Sorts the array using bubble sort.

    :param arr: the array to be sorted
    :return: the sorted array
    """
    n = len(arr)
    for i in range(n):
        for j in range(0, n-i-1):
            if arr[j] > arr[j+1]:
                arr[j], arr[j+1] = arr[j+1], arr[j]
    return arr
----------------------------------------

📝 Test 2:
Code:
def fibonacci(n):
    if n <= 1:
        return n
    return fibonacci(n-1) + fibonacci(n-2)

Generated Documentation:
Fibonacci sequence.

    :param n: The nth number in the sequence.
    :return: The nth number in the sequence.
    """
    if n <= 1:
        return n
    return fibonacci(n-1) + fibonacci(n-2)
----------------------------------------

📝 Test 3:
Code:
de